In [ ]:
!pip install mediapipe

# 1. Import

In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# 2. Keypoints using Mediapipe Holistic

In [2]:
mp_hands = mp.solutions.hands
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils # drawing utils
mp_drawing_styles = mp.solutions.drawing_styles

In [8]:
cap = cv2.VideoCapture(0)
with mp_hands.Hands(model_complexity=0,min_detection_confidence=0.5,min_tracking_confidence=0.5) as hands:
    with mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            success, image = cap.read()
            
            if not success:
                print("Ignoring empty camera frame.")
                # If loading a video, use 'break' instead of 'continue'.
                continue

            # To improve performance, optionally mark the image as not writeable to
            # pass by reference.
            image.flags.writeable = False
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = hands.process(image)
            results_pose = pose.process(image)
            

            # Draw the hand annotations on the image.
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            
            hand_np = []
            if results.multi_hand_landmarks:
                for i in range(len(results.multi_hand_landmarks)):
                    print(i)
                    mp_drawing.draw_landmarks(image,results.multi_hand_landmarks[i],mp_hands.HAND_CONNECTIONS,mp_drawing_styles.get_default_hand_landmarks_style(),mp_drawing_styles.get_default_hand_connections_style())
                    hand_np.append(np.array([[res.x, res.y, res.z] for res in results.multi_hand_landmarks[i].landmark]).flatten() if results.multi_hand_landmarks[i] else np.zeros(21*3))
                    
            if len(hand_np) == 0:
                lh = np.zeros(21*3)
                rh = np.zeros(21*3)
            elif len(hand_np) == 1:
                lh = hand_np[0]
                rh = np.zeros(21*3)
            else:
                lh = hand_np[0]
                rh = hand_np[1]        
                    
            # Draw pose annotations        
            mp_drawing.draw_landmarks(image,results_pose.pose_landmarks,mp_pose.POSE_CONNECTIONS,landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
            
            ps = np.array([[res.x, res.y, res.z, res.visibility] for res in results_pose.pose_landmarks.landmark]).flatten() if results_pose.pose_landmarks else np.zeros(33*4)
            
            extracted_keypoints = np.concatenate([ps, lh, rh])
            print("EXTRACTED_KEYPOINTS: ")
            print(len(extracted_keypoints))
            
            # Flip the image horizontally for a selfie-view display.
            cv2.imshow('MediaPipe Result', cv2.flip(image, 1))
            if cv2.waitKey(10) & 0xFF == ord('q'):
                  break
cap.release()
cap.destroyAllWindows()

EXTRACTED_KEYPOINTS: 
258


error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1267: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'


# 3. Extract Keypoints Values

In [3]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

# 4. Setup Folders for Manual Collection

In [4]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data_m_noface') 

# Actions that we try to detect
actions = ['can','you','help','me','what','name','hamburger','french fries','thanks','bye','hello','excuse me','sorry']

In [5]:
# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

In [6]:
len(actions)

16

In [19]:
np.save('actions_list.npy',actions)

In [13]:
os.makedirs('MP_Data_m_noface',exist_ok=True)
for action in actions:
    os.makedirs(os.path.join(DATA_PATH, action),exist_ok=True)

In [14]:
for action in actions: 
    for sequence in range(1,no_sequences+1):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

# 5. Manual data collecting

In [18]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        ret, frame = cap.read()           
        cv2.putText(frame, 'Moving on to the next sign: {}, you have 30 seconds to learn the next sign'.format(action), (15,12), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
        # Show to screen
        cv2.imshow('OpenCV Feed', frame)
        cv2.waitKey(30000)
        
        for sequence in range(1,no_sequences+1):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(500)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    cap.release()
                    cv2.destroyAllWindows()
                    break
                    
                    
    cap.release()
    cv2.destroyAllWindows()

# 6. Use downloaded Dataset

In [ ]:
root = 'C:\\Users\\ryanr\\ASLtoEng\\'
MP_DATA_PATH = os.path.join(root,'MP_Data_MSASL')

In [ ]:
for label in os.listdir(MP_DATA_PATH):
    LABEL_PATH = os.path.join(MP_DATA_PATH,label)
    for filename in os.listdir(LABEL_PATH):
        if filename.endswith('.mp4'):
            print(f'Reading {filename}...')
            
            video_name = filename.split('.')[0]
            VIDEO_PATH = os.path.join(LABEL_PATH,filename)
            VIDEO_FOLDER_PATH = os.path.join(LABEL_PATH,video_name)
#             VIDEO_FOLDER_PATH_FLIPPED = os.path.join(LABEL_PATH,video_name+"(1)")
            os.makedirs(VIDEO_FOLDER_PATH,exist_ok=True)
#             os.makedirs(VIDEO_FOLDER_PATH_FLIPPED,exist_ok=True)
            
            # Capture video
            vidcap = cv2.VideoCapture(VIDEO_PATH)
            # Set mediapipe model 
            with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

                # Read feed
                success, frame = vidcap.read()
                
                frame_num = 0
                
                while success:
                    # Make detections
                    image, results = mediapipe_detection(frame, holistic)
#                     image_flipped, results_flipped = mediapipe_detection(cv2.flip(frame, 1), holistic)
#                     image_flipped, results_flipped = mediapipe_detection(frame, holistic)

                    
#                     Draw detections for debugging
#                     draw_styled_landmarks(image, results)
#                     cv2.imshow('OpenCV Feed', image)
#                     cv2.waitKey(10)         
            
#                     draw_styled_landmarks(image_flipped, results_flipped)
#                     cv2.imshow('OpenCV Feed', image_flipped)
                    
                    # Export keypoints
                    keypoints = extract_keypoints(results)
                    npy_path = os.path.join(VIDEO_FOLDER_PATH, str(frame_num))
                    np.save(npy_path, keypoints)
                    
#                     keypoints_flipped = extract_keypoints(results_flipped)
#                     npy_path_flipped = os.path.join(VIDEO_FOLDER_PATH_FLIPPED, str(frame_num))
#                     np.save(npy_path_flipped, keypoints_flipped)
                         
                    success,frame = vidcap.read()
                    frame_num += 1
            
                    Break gracefully
                    if cv2.waitKey(10) & 0xFF == ord('q'):
                        break
                
                print('Converted.')                    
                        
                Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
            
            # close the video
            vidcap.release()
            
cv2.destroyAllWindows()        

In [ ]:
import shutil

In [ ]:
# remove videos after converting
for label in os.listdir(MP_DATA_PATH):
    LABEL_PATH = os.path.join(MP_DATA_PATH,label)
    for filename in os.listdir(LABEL_PATH):
        if filename.endswith('.mp4'):
#         if filename.startswith('fast_') and filename.endswith('(1)'):
            filepath = os.path.join(LABEL_PATH,filename)
            print(f'Deleting {filename}...')
            os.remove(filepath)
#             shutil.rmtree(filepath)

In [ ]:
# sanity check that the data collected is correct
for label in os.listdir(MP_DATA_PATH):
    LABEL_PATH = os.path.join(MP_DATA_PATH,label)
    for filename in os.listdir(LABEL_PATH):
        if not filename.endswith('.mp4'):
            for npfile in os.listdir(os.path.join(LABEL_PATH,filename)):
                print(np.load(os.path.join(LABEL_PATH,filename,npfile)))
                break

# 6.* Preprocess Data and Create Labels and Features

In [6]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [ ]:
label_map = {label:num for num, label in enumerate(actions)}
print(label_map)

In [ ]:
sequences, labels = [], []
for action in label_map.keys():
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))):
        window = []
        last_frame = 0
        for frame_num in range(sequence_length):
            if  os.path.exists(os.path.join(DATA_PATH, action, sequence, "{}.npy".format(frame_num))):
                res = np.load(os.path.join(DATA_PATH, action, sequence, "{}.npy".format(frame_num)))
                last_frame = frame_num
            else:
#                 res = np.zeros((1662,),dtype='float64') # zero padding
                res = np.load(os.path.join(DATA_PATH, action, sequence, "{}.npy".format(last_frame))) # last frame padding
            print(len(np.concatenate([res[0:132],res[1536:]]))) 
            window.append(np.concatenate([res[0:132],res[1536:]])) # skip face landmarks
        sequences.append(window)
        labels.append(label_map[action])        

In [13]:
X = np.array(sequences)

In [14]:
X.shape

(480, 30, 258)

In [ ]:
np.save('cached_data/X_data_m_noface',X)

In [15]:
y = to_categorical(labels,num_classes=len(actions)).astype(int)

In [ ]:
np.save('cached_data/y_data_m_noface',y)

In [16]:
y.shape

(480, 16)

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=y)

In [18]:
y_train.sum(axis=0)

array([27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27])

# 7. Build and Train LSTM Neural Network

In [19]:
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import ModelCheckpoint

In [20]:
log_dir = os.path.join('Logs/data_m_noface')
tb_callback = TensorBoard(log_dir=log_dir)

In [21]:
checkpoint_path = os.path.join("Logs","data_m_noface",'checkpoints','cp-{epoch:04d}.ckpt')
cp_callback = ModelCheckpoint(filepath=checkpoint_path,verbose=1,save_weights_only=True,save_freq=50*14)

In [ ]:
model = Sequential()
model.add(LSTM(16, return_sequences=True, activation='relu', input_shape=(sequence_length,258)))
model.add(LSTM(32, return_sequences=True, activation='relu'))
model.add(LSTM(16, return_sequences=False, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(len(actions), activation='softmax'))

In [24]:
# opt = SGD(learning_rate=0.01,momentum=0.9)
# opt = Adam(learning_rate=0.0001)
opt = 'Adam'

In [25]:
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [31]:
model.load_weights('Logs/data_m_noface/checkpoints/cp-0292.ckpt')

In [26]:
model.fit(X_train, y_train, epochs=2000, callbacks=[cp_callback])

Epoch 1/2000
14/14 [==============================] - 5s 179ms/step - loss: 2.7773 - categorical_accuracy: 0.0486
Epoch 2/2000
14/14 [==============================] - 3s 182ms/step - loss: 2.7653 - categorical_accuracy: 0.0694
Epoch 3/2000
14/14 [==============================] - 3s 185ms/step - loss: 2.7350 - categorical_accuracy: 0.0787
Epoch 4/2000
14/14 [==============================] - 3s 198ms/step - loss: 2.6344 - categorical_accuracy: 0.0972
Epoch 5/2000
14/14 [==============================] - 3s 204ms/step - loss: 2.4364 - categorical_accuracy: 0.1713
Epoch 6/2000
14/14 [==============================] - 3s 198ms/step - loss: 4.2146 - categorical_accuracy: 0.1343
Epoch 7/2000
14/14 [==============================] - 3s 200ms/step - loss: 2.7680 - categorical_accuracy: 0.0556
Epoch 8/2000
14/14 [==============================] - 3s 209ms/step - loss: 2.6852 - categorical_accuracy: 0.0787
Epoch 9/2000
14/14 [==============================] - 3s 223ms/step - loss: 2.5465 - cat

14/14 [==============================] - 3s 203ms/step - loss: 0.5306 - categorical_accuracy: 0.7639
Epoch 140/2000
 3/14 [=====>........................] - ETA: 2s - loss: 0.5531 - categorical_accuracy: 0.7292
Epoch 140: saving model to Logs\data_m_noface\checkpoints\cp-0140.ckpt
14/14 [==============================] - 3s 221ms/step - loss: 0.5004 - categorical_accuracy: 0.7708
Epoch 141/2000
14/14 [==============================] - 3s 205ms/step - loss: 0.5295 - categorical_accuracy: 0.7662
Epoch 142/2000
14/14 [==============================] - 3s 207ms/step - loss: 0.5883 - categorical_accuracy: 0.7454
Epoch 143/2000
14/14 [==============================] - 3s 215ms/step - loss: 0.5660 - categorical_accuracy: 0.7616
Epoch 144/2000
14/14 [==============================] - 3s 224ms/step - loss: 0.4759 - categorical_accuracy: 0.7870
Epoch 145/2000
14/14 [==============================] - 3s 209ms/step - loss: 0.4356 - categorical_accuracy: 0.8125
Epoch 146/2000
14/14 [===============

14/14 [==============================] - 3s 189ms/step - loss: 0.0984 - categorical_accuracy: 0.9699
Epoch 276/2000
14/14 [==============================] - 3s 204ms/step - loss: 0.0833 - categorical_accuracy: 0.9792
Epoch 277/2000
14/14 [==============================] - 3s 206ms/step - loss: 0.0492 - categorical_accuracy: 0.9907
Epoch 278/2000
14/14 [==============================] - 3s 196ms/step - loss: 0.0806 - categorical_accuracy: 0.9699
Epoch 279/2000
 7/14 [==============>...............] - ETA: 1s - loss: 0.0638 - categorical_accuracy: 0.9821
Epoch 279: saving model to Logs\data_m_noface\checkpoints\cp-0279.ckpt
14/14 [==============================] - 3s 215ms/step - loss: 0.0724 - categorical_accuracy: 0.9769
Epoch 280/2000
14/14 [==============================] - 3s 196ms/step - loss: 0.4474 - categorical_accuracy: 0.8519
Epoch 281/2000
14/14 [==============================] - 3s 197ms/step - loss: 0.5294 - categorical_accuracy: 0.8426
Epoch 282/2000
14/14 [===============

14/14 [==============================] - 3s 211ms/step - loss: 0.1022 - categorical_accuracy: 0.9722
Epoch 412/2000
14/14 [==============================] - 3s 225ms/step - loss: 0.0782 - categorical_accuracy: 0.9838
Epoch 413/2000
14/14 [==============================] - 3s 206ms/step - loss: 0.0554 - categorical_accuracy: 0.9861
Epoch 414/2000
14/14 [==============================] - 3s 204ms/step - loss: 0.0406 - categorical_accuracy: 0.9931
Epoch 415/2000
14/14 [==============================] - 3s 190ms/step - loss: 0.0983 - categorical_accuracy: 0.9792
Epoch 416/2000
14/14 [==============================] - 3s 192ms/step - loss: 0.0888 - categorical_accuracy: 0.9699
Epoch 417/2000
14/14 [==============================] - 3s 204ms/step - loss: 0.0734 - categorical_accuracy: 0.9815
Epoch 418/2000
11/14 [======================>.......] - ETA: 0s - loss: 0.0564 - categorical_accuracy: 0.9801
Epoch 418: saving model to Logs\data_m_noface\checkpoints\cp-0418.ckpt
14/14 [===============

14/14 [==============================] - 3s 217ms/step - loss: 3.3271e-04 - categorical_accuracy: 1.0000
Epoch 546/2000
14/14 [==============================] - 3s 203ms/step - loss: 3.4457e-04 - categorical_accuracy: 1.0000
Epoch 547/2000
14/14 [==============================] - 3s 220ms/step - loss: 5.1269e-04 - categorical_accuracy: 1.0000
Epoch 548/2000
14/14 [==============================] - 3s 214ms/step - loss: 9.5666e-04 - categorical_accuracy: 1.0000
Epoch 549/2000
14/14 [==============================] - 3s 204ms/step - loss: 4.3448e-04 - categorical_accuracy: 1.0000
Epoch 550/2000
14/14 [==============================] - 3s 218ms/step - loss: 3.4743e-04 - categorical_accuracy: 1.0000
Epoch 551/2000
14/14 [==============================] - 3s 225ms/step - loss: 3.1153e-04 - categorical_accuracy: 1.0000
Epoch 552/2000
14/14 [==============================] - 3s 206ms/step - loss: 3.0295e-04 - categorical_accuracy: 1.0000
Epoch 553/2000
14/14 [==============================] -

KeyboardInterrupt: 

In [ ]:
model.summary()

# 8. Make Predictions

In [ ]:
res = model.predict(X_test)
res.shape

In [ ]:
actions[np.argmax(res[2])]

In [ ]:
actions[np.argmax(y_test[2])]

# 9. Save Weights

In [ ]:
model.save('weights/data_m_noface.h5')

In [ ]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

In [ ]:
del model

In [ ]:
# Save keras model & weights to saved_model folder
tensorflow.saved_model.save(model,'saved_model/1')
# Use this command to convert into onnx to deploy !python -m tf2onnx.convert --saved-model C:\Users\ryanr\ASLtoEng\saved_model\1 --output model.onnx

In [ ]:
model.load_weights('weights/data_m_noface.h5')

# 10. Evaluation using Confusion Matrix and Accuracy

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [ ]:
yhat = model.predict(X_test)

In [ ]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [ ]:
multilabel_confusion_matrix(ytrue, yhat)

In [ ]:
accuracy_score(ytrue, yhat)

# 11. Test in Real Time

In [ ]:
from scipy import stats

In [25]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num%3], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [ ]:
plt.figure(figsize=(18,18))
plt.imshow(prob_viz(res, actions, image, colors))

In [ ]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(1)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = hands.process(image)
        results_pose = pose.process(image)


        # Draw the hand annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)



#             print(len(results.multi_hand_landmarks))
        hand_np = []
        if results.multi_hand_landmarks:
            for i in range(len(results.multi_hand_landmarks)):
                print(i)
                mp_drawing.draw_landmarks(image,results.multi_hand_landmarks[i],mp_hands.HAND_CONNECTIONS,mp_drawing_styles.get_default_hand_landmarks_style(),mp_drawing_styles.get_default_hand_connections_style())
                hand_np.append(np.array([[res.x, res.y, res.z] for res in results.multi_hand_landmarks[i].landmark]).flatten() if results.multi_hand_landmarks[i] else np.zeros(21*3))

        if len(hand_np) == 0:
            lh = np.zeros(21*3)
            rh = np.zeros(21*3)
        elif len(hand_np) == 1:
            lh = hand_np[0]
            rh = np.zeros(21*3)
        else:
            lh = hand_np[0]
            rh = hand_np[1]        

        # Draw pose annotations        
        mp_drawing.draw_landmarks(image,results_pose.pose_landmarks,mp_pose.POSE_CONNECTIONS,landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())

        ps = np.array([[res.x, res.y, res.z, res.visibility] for res in results_pose.pose_landmarks.landmark]).flatten() if results_pose.pose_landmarks else np.zeros(33*4)

        keypoints = np.concatenate([ps, lh, rh])
        sequence.append(keypoints)
        sequence = sequence[-sequence_length:]
        
        if len(sequence) == sequence_length:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
            #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 4: 
                sentence = sentence[-4:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()